In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import re

In [2]:
!pwd

/mnt/d/data_science/Insurance_report/visualization


In [3]:
def remove_chinese_sign(data):
    columns = data.columns.to_list()
    columns = [re.sub('_\d、','',x) for x in columns]
    columns = [re.sub('_（\d）','',x) for x in columns]
    columns = [re.sub('：',':',x) for x in columns]
    data.columns = columns
    return data

In [4]:
industry_old = pd.read_csv('../data/industry_old.csv')
industry_old = remove_chinese_sign(industry_old)
len(industry_old.columns)

28

In [5]:
industry_old['银行存款'] = industry_old['其中:银行存款'].dropna().append(industry_old['银行存款'].dropna())

In [6]:
industry_old['投资'] = (industry_old['债券'] + industry_old['股票和证券投资基金']).dropna().append(industry_old['投资'].dropna())

In [7]:
industry_old['Year'] = industry_old['info'].str.findall(r'\d{4}(?=年)').map(lambda x: x[0]).astype(int)

In [8]:
industry_old['Month'] = industry_old['info'].str.findall(r'(?<=年).*(?=保险业经营情况)').map(lambda x:x[0]).replace({'':'1-12月'}).map(lambda x:x[:-1])

In [9]:
numerical_cols = industry_old.select_dtypes(float).columns

In [10]:
industry_old.loc[industry_old.Year == 2019,numerical_cols] *= 10000

In [11]:
industry_old.shape

(17, 30)

In [12]:
industry_data = pd.read_csv('../data/industry_data.csv')
industry_data = remove_chinese_sign(industry_data)
len(industry_data.columns)

23

In [17]:
industry_data.shape

(107, 23)

In [ ]:
common = set(industry_old.columns).intersection(set(industry_data.columns))

In [ ]:
industry_old.columns

In [ ]:
industry_data.columns

In [13]:
industry_old[['from_url','养老保险公司企业年金缴费', '养老保险公司企业年金受托管理资产', '养老保险公司企业年金投资管理资产']] = np.nan

In [14]:
combined = industry_data.copy()

In [15]:
combined = combined.append(industry_old.loc[:,industry_data.columns])

In [16]:
combined.shape

(124, 23)

In [ ]:
combined.sort_values(['Year','Month'])

In [18]:
industry_new = pd.read_csv('../data/industry_new.csv')
industry_data = remove_chinese_sign(industry_new)
industry_new.shape

(22, 24)

In [ ]:
len(industry_new.columns)

In [ ]:
set(combined).intersection(set(industry_new))

In [ ]:
len(combined.columns)

In [19]:
numerical_cols = industry_new.select_dtypes(float).columns

In [20]:
industry_new.loc[:,numerical_cols] *= 10000

In [21]:
industry_new = industry_new.rename(columns = {'其中:银行存款':'银行存款'})

In [22]:
industry_new['投资'] = industry_new['债券'] + industry_new['股票和证券投资基金']

In [23]:
industry_new[['from_url','养老保险公司企业年金缴费','养老保险公司企业年金受托管理资产', '养老保险公司企业年金投资管理资产','人身保险公司保户投资款新增交费', '人身保险公司投连险独立账户新增交费']] = np.nan

In [24]:
combined = combined.append(industry_new.loc[:,combined.columns])

In [25]:
combined.shape

(146, 23)

In [43]:
x = {f'年{i}月':f'年1-{i}月' for i in range(2,13)}
combined['Year'] = combined['info'].str.findall(r'\d{4}(?=年)').map(lambda x: x[0]).astype(int)
combined['Month'] = combined['info'].str.findall(r'(?=年).*(?=保险业.*)').map(lambda x:x[0]).replace({'年':'年1-12月',**x}).map(lambda x: x[1:-1])
combined.Month.unique()

array(['1-2', '1', '1-12', '1-11', '1-10', '1-9', '1-8', '1-7', '1-6',
       '1-5', '1-4', '1-3'], dtype=object)

In [44]:
month_cat = ['1'] + [f'1-{i}' for i in range(2,13)]
combined['Month'] = pd.Categorical(combined['Month'],ordered = True,categories=month_cat)   

In [ ]:
len(industry_old.columns)

In [ ]:
combined['Month'] = pd.Categorical(combined['Month'],ordered = True,categories=month_cat)      

In [ ]:
combined.info()

In [ ]:
combined = combined.sort_values(['Year','Month'])

In [ ]:
combined.groupby('Year')['原保险保费收入'].count()

In [47]:
combined = combined.sort_values(['Year','Month'])

In [49]:
combined.groupby('Year')['原保险保费收入'].count()

Year
2009    12
2010    12
2011    10
2012    12
2013    12
2014    12
2015    11
2016    12
2017    13
2018    14
2019     4
2020    12
2021    10
Name: 原保险保费收入, dtype: int64

In [51]:
combined.to_csv('../data/industry_combined.csv',index = False)